In [1]:
import pandas as pd
import numpy as np
import time
from sklearn.impute import SimpleImputer

# 数据预处理

In [2]:
imp = SimpleImputer(missing_values=np.nan, strategy='most_frequent')

In [3]:
df = pd.read_csv('./clean_data/xgb_fraud_with_magic_train.csv')

In [4]:
# data=np.array(df)
# print(np.all(np.isfinite(data)))
# imp.fit(data)
# data = imp.fit_transform(data)

In [36]:
df.fillna(-1)

,TransactionID,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,...,D9_card1_addr1_P_emaildomain_mean,D9_card1_addr1_P_emaildomain_std,D11_card1_mean,D11_card1_std,D11_card1_addr1_mean,D11_card1_addr1_std,D11_card1_addr1_P_emaildomain_mean,D11_card1_addr1_P_emaildomain_std,isFraud,DT_M
0,2987000.0,86400.0,68.500000,4,12928.0,-1.0,50.0,1,42.0,1,...,-1.000000,-1.000000,288.00,71.875,408.00,-1.000,408.00,-1.00000,0,12
1,2987001.0,86401.0,29.000000,4,1755.0,304.0,50.0,2,2.0,1,...,0.770996,0.029465,394.75,243.875,275.25,172.375,357.75,252.50000,0,12
2,2987002.0,86469.0,59.000000,4,3664.0,390.0,50.0,3,66.0,2,...,-1.000000,-1.000000,301.75,184.625,388.75,197.375,710.00,0.19751,0,12
3,2987003.0,86499.0,50.000000,4,17136.0,467.0,50.0,2,17.0,2,...,0.750000,-1.000000,393.75,242.000,428.50,218.000,464.00,184.37500,0,12
4,2987004.0,86506.0,50.000000,1,3496.0,414.0,50.0,2,2.0,1,...,-1.000000,-1.000000,432.50,245.125,-1.00,-1.000,-1.00,-1.00000,0,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
590535,3577535.0,15811047.0,49.000000,4,5552.0,-1.0,50.0,3,126.0,2,...,-1.000000,-1.000000,354.75,217.500,326.25,170.875,325.75,137.37500,0,17
590536,3577536.0,15811049.0,39.500000,4,9440.0,125.0,50.0,2,124.0,2,...,-1.000000,-1.000000,448.75,293.750,448.75,293.750,104.00,62.68750,0,17
590537,3577537.0,15811079.0,30.953125,4,11040.0,495.0,50.0,2,124.0,2,...,0.471436,0.277588,382.50,234.250,387.00,216.625,339.00,206.12500,0,17
590538,3577538.0,15811088.0,117.000000,4,6824.0,381.0,50.0,2,124.0,2,...,0.708496,0.125000,405.50,234.375,393.25,238.125,301.50,152.62500,0,17


In [37]:
def reduce_mem_usage(df):
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

In [38]:
import datetime
START_DATE = datetime.datetime.strptime('2017-11-30', '%Y-%m-%d')
df['DT_M'] = df['TransactionDT'].apply(lambda x: (START_DATE + datetime.timedelta(seconds = x)))
df['DT_M'] = (df['DT_M'].dt.year-2017)*12 + df['DT_M'].dt.month 

y = df['isFraud'].copy()


In [39]:
train_cols = ['TransactionAmt', 'ProductCD', 'card1', 'card2', 'card3', 'card5',
       'card6', 'addr1', 'addr2', 'dist1', 'dist2', 'P_emaildomain',
       'R_emaildomain', 'C1', 'C2', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9',
       'C10', 'C11', 'C12', 'C13', 'C14', 'D1', 'D2', 'D3', 'D4', 'D5',
       'D10', 'D11', 'D15', 'M1', 'M2', 'M3', 'M4', 'M6', 'M7', 'M8',
       'M9', 'V1', 'V3', 'V4', 'V6', 'V8', 'V11', 'V13', 'V14', 'V17',
       'V20', 'V23', 'V26', 'V27', 'V30', 'V36', 'V37', 'V40', 'V41',
       'V44', 'V47', 'V48', 'V54', 'V56', 'V59', 'V62', 'V65', 'V67',
       'V68', 'V70', 'V76', 'V78', 'V80', 'V82', 'V86', 'V88', 'V89',
       'V91', 'V107', 'V108', 'V111', 'V115', 'V117', 'V120', 'V121',
       'V123', 'V124', 'V127', 'V129', 'V130', 'V136', 'V138', 'V139',
       'V142', 'V147', 'V156', 'V160', 'V162', 'V165', 'V166', 'V169',
       'V171', 'V173', 'V175', 'V176', 'V178', 'V180', 'V182', 'V185',
       'V187', 'V188', 'V198', 'V203', 'V205', 'V207', 'V209', 'V210',
       'V215', 'V218', 'V220', 'V221', 'V223', 'V224', 'V226', 'V228',
       'V229', 'V234', 'V235', 'V238', 'V240', 'V250', 'V252', 'V253',
       'V257', 'V258', 'V260', 'V261', 'V264', 'V266', 'V267', 'V271',
       'V274', 'V277', 'V281', 'V283', 'V284', 'V285', 'V286', 'V289',
       'V291', 'V294', 'V296', 'V297', 'V301', 'V303', 'V305', 'V307',
       'V309', 'V310', 'V314', 'V320', 'id_01', 'id_02', 'id_03', 'id_04',
       'id_05', 'id_06', 'id_09', 'id_10', 'id_11', 'id_12', 'id_13',
       'id_15', 'id_16', 'id_17', 'id_18', 'id_19', 'id_20', 'id_28',
       'id_29', 'id_31', 'id_35', 'id_36', 'id_37', 'id_38', 'DeviceType',
       'DeviceInfo', 'cents', 'addr1_FE', 'card1_FE', 'card2_FE',
       'card3_FE', 'P_emaildomain_FE', 'card1_addr1',
       'card1_addr1_P_emaildomain', 'card1_addr1_FE',
       'card1_addr1_P_emaildomain_FE', 'TransactionAmt_card1_mean',
       'TransactionAmt_card1_std', 'TransactionAmt_card1_addr1_mean',
       'TransactionAmt_card1_addr1_std',
       'TransactionAmt_card1_addr1_P_emaildomain_mean',
       'TransactionAmt_card1_addr1_P_emaildomain_std', 'D9_card1_mean',
       'D9_card1_std', 'D9_card1_addr1_mean', 'D9_card1_addr1_std',
       'D9_card1_addr1_P_emaildomain_mean',
       'D9_card1_addr1_P_emaildomain_std', 'D11_card1_mean',
       'D11_card1_std', 'D11_card1_addr1_mean', 'D11_card1_addr1_std',
       'D11_card1_addr1_P_emaildomain_mean',
       'D11_card1_addr1_P_emaildomain_std']

In [40]:
%%time
df = reduce_mem_usage(df)
X = df[train_cols]
y = y.astype(np.int8)

Memory usage of dataframe is 281.03 MB
Memory usage after optimization is: 277.09 MB
Decreased by 1.4%
CPU times: user 4.25 s, sys: 124 ms, total: 4.37 s
Wall time: 4.36 s


In [41]:
# TRAIN 75% PREDICT 25%
idxT = df.index[:3*len(df)//4]
idxV = df.index[3*len(df)//4:]

# 模型

In [42]:
from sklearn.model_selection import KFold
from sklearn import svm
from sklearn.metrics import roc_auc_score
from sklearn import linear_model
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate

import lightgbm as lgb
import torch.optim as optim
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier

In [43]:
class My_net(nn.Module):
    def __init__(self):
        super(My_net,self).__init__()
        self.fc1 = nn.Linear(336, 1024)
        self.relu1 = nn.ReLU()
        self.dout1 = nn.Dropout(0.25)
        self.fc2 = nn.Linear(1024, 2048)
        self.relu2 = nn.ReLU()
        self.dout2 = nn.Dropout(0.25)
        self.fc3 = nn.Linear(2048, 512)
        self.relu3 = nn.ReLU()
        self.dout3 = nn.Dropout(0.2)
        self.fc4 = nn.Linear(512, 64)
        self.prelu = nn.PReLU(1)
        self.out = nn.Linear(64, 2)
        self.out_act = nn.Softmax(dim=1)
        
    def forward(self, input_):
        a1 = self.fc1(input_)
        h1 = self.relu1(a1)
        dout1 = self.dout1(h1)
        a2 = self.fc2(dout1)
        h2 = self.relu2(a2)
        dout2 = self.dout2(h2)
        a3 = self.fc3(dout2)
        h3 = self.relu3(a3)
        dout3 = self.dout3(h3)
        a4 = self.fc4(dout3)
        h4 = self.prelu(a4)
        a5 = self.out(h4)
        y = self.out_act(a5)
        return y

In [44]:
all_clf = [
    ('svc', svm.SVC(C=0.1, gamma=0.001, kernel='rbf', probability=True)),
    ('lr', linear_model.LogisticRegressionCV(Cs=0.01, penalty='l2',class_weight='balanced',solver='liblinear', n_jobs=24)),
    ('gnb', GaussianNB(priors=[None,[0.97,0.03],[0.9,0.1]])),
    ('DT',DecisionTreeClassifier(max_features=100, max_depth=12)),
    ('LightGBM', lgb.LGBMClassifier(learning_rate=0.01, reg_alpha=1, reg_lambda=1, eval_metric='auc', nthread=24,
            tree_method='hist')),
    ('xgb', XGBClassifier(learning_rate=0.01, reg_alpha=1, reg_lambda=1, eval_metric='auc', n_jobs=24,
            tree_method='hist'))
]

In [45]:
all_params = [
    {'C':0.1, 'gamma':0.001, 'kernel':'rbf'},
    {'C':0.01, 'penalty':'l2'},
    {'priors':[None,[0.97,0.03],[0.9,0.1]]},
    {'max_features':100, 'max_depth':12},
    {'learning_rate':0.01, 'reg_alpha':1, 'reg_lambda':1, 'eval_metric':'auc', 'nthread':24,
            'tree_method':'hist'},
    {'learning_rate':0.01, 'reg_alpha':1, 'reg_lambda':1, 'eval_metric':'auc', 'nthread':24,
            'tree_method':'hist'}
]

In [35]:
MLA_columns = ['MLA Name', 'Time', 'Train Accuracy Mean', 'Test Accuracy Mean']
MLA_compare = pd.DataFrame(columns=MLA_columns)
row_index = 0
for clf, param in zip (all_clf, all_params):
    cv_results = cross_validate(clf[1], X, y, scoring='roc_auc', cv=[(idxT, idxV)], n_jobs=12, return_train_score=True)
    print(sorted(cv_results.keys()))
    MLA_compare.loc[row_index, 'MLA Name'] = clf[1].__class__.__name__
    MLA_compare.loc[row_index, 'Time'] = cv_results['fit_time'].mean()
    MLA_compare.loc[row_index, 'Train Accuracy Mean'] = cv_results['train_score'].mean()
    MLA_compare.loc[row_index, 'Test Accuracy Mean'] = cv_results['test_score'].mean()
#     y_preds = clf[1].predict_proba(idxV)
#     y_preds.to_csv('./')
    row_index+=1


['fit_time', 'score_time', 'test_score', 'train_score']
['fit_time', 'score_time', 'test_score', 'train_score']
['fit_time', 'score_time', 'test_score', 'train_score']
['fit_time', 'score_time', 'test_score', 'train_score']


In [ ]:
MLA_compare